## Instruction SFT Model

Rinna 4b: https://huggingface.co/rinna/bilingual-gpt-neox-4b-instruction-sft

In [ ]:
!pip install transformers accelerate sentencepiece peft bitsandbytes einops --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.1 MB/s eta 0:00:00


In [ ]:
# Wrap printed text
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
model_name = "rinna/bilingual-gpt-neox-4b-instruction-sft"

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer.vocab_size

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


65536

In [ ]:
%time

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map="auto",
)

# unnecessary when quantized
# if torch.cuda.is_available():
#     model = model.to("cuda")

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 7.39 µs


In [ ]:
prompt = [
    {
        "speaker": "ユーザー",
        "text": "You are a smart, senior business consultant with excellent thinking and communication skills. You always answer in Japanese."
    },
    {
        "speaker": "システム",
        "text": "はい、よろしくお願いいたします。今回はどのようにお手伝いできますでしょうか？"
    },
    {
        "speaker": "ユーザー",
        "text": "新規事業を始める際の一番効果的なステップを教えてください。"
    }
]
prompt = [
    f"{uttr['speaker']}: {uttr['text']}"
    for uttr in prompt
]
prompt = "\n".join(prompt)
prompt = (
    prompt
    + "\n"
    + "システム: "
)
print(prompt)

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=512,
        do_sample=True,
        temperature=1.0,
        top_p=0.85,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
print(output)

うーん、これは本当に難しい質問ですね。新規事業を始めるのは簡単ではありませんが、最も効果的な方法は、適切なリソースとスキルを持つ外部の協力者を見つけることです。これには、市場調査を行う人や広告会社の人々、ビジネスプランを作成する人、顧客基盤を築くために協力してくれる人々の支援などがあります。さらに、リソースやスキルが不足している場合は、ビジネスプランを作成する際に支援を求めてください。つまり、あなたは外部のリソースやアイデアを求めるための手助けが必要です。これは、あなたのアイデアやビジネスプランについて意見を聞くために、支援する側の人々と行うことができます。

これらのリソースや経験がない場合は、支援を求めてください。あなたがどのようにしてこのようなリソースや経験を得たかについて、より詳しくお話しすることができます。</s>


In [ ]:
text = """
りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？
""".strip()
inputs = tokenizer(f"ユーザー: {text}\nシステム: 一歩ずつ考えましょう。", return_tensors='pt', truncation=True, add_special_tokens=False)

model.eval()
with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_length=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        top_k=0,
        repetition_penalty=1.1,
        num_beams=1,
        pad_token_id=tokenizer.pad_token_id,
        num_return_sequences=1,
    )

output = tokenizer.decode(output_ids[0], skip_special_tokens=False)
print(output)

ユーザー: りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう?
システム: 一歩ずつ考えましょう。まず、りんごを1つずつ取り除くと、5-2=3個のりんごがあります。次に、りんごを取り除く前に持っていたりんごの数を加算すると、3+2=5個のりんごがありました。したがって、答えは5です。</s>


In [ ]:
text = """バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)


ユーザー: バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう?
システム: バットの価格は1100-1000=200円高くなります。したがって、ボールの価格は200-1000=50ドル高くなります。最終的な答え:50。


In [ ]:
text = """
引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)


ユーザー: 引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
システム: k=3


In [ ]:
text = """
下記の英語を日本語に翻訳してください。
English: There were 3 apples and 2 oranges. How many fruits were there in total?
""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


ユーザー: 下記の英語を日本語に翻訳してください。
English: There were 3 apples and 2 oranges. How many fruits were there in total?
システム: There were 3 apples, 2 oranges and 1 grapefruit.</s>


In [ ]:
text = """
下記の文章を100字以内に要約してください。

```
【速報】ロシア月探査機「ルナ25」が月に衝突 「消滅した」ロスコスモス発表
月に向かっていたロシアの無人探査機「ルナ25」が月に衝突したことが分かった。ロシアの国営宇宙企業ロスコスモスは先ほど、「月に衝突し、消滅した」と明らかにした。月面着陸前の軌道に移行中、制御不能となったという。
探査機は21日に月の南極付近に着陸予定だった。
```
""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


ユーザー: 下記の文章を100字以内に要約してください。

```
【速報】ロシア月探査機「ルナ25」が月に衝突 「消滅した」ロスコスモス発表
月に向かっていたロシアの無人探査機「ルナ25」が月に衝突したことが分かった。ロシアの国営宇宙企業ロスコスモスは先ほど、「月に衝突し、消滅した」と明らかにした。月面着陸前の軌道に移行中、制御不能となったという。
探査機は21日に月の南極付近に着陸予定だった。
```
システム: ロシアの月探査機が月に衝突</s>


In [ ]:
text = """
下記の文章をJSON形式に変換してください。

```
【速報】ロシア月探査機「ルナ25」が月に衝突 「消滅した」ロスコスモス発表
月に向かっていたロシアの無人探査機「ルナ25」が月に衝突したことが分かった。ロシアの国営宇宙企業ロスコスモスは先ほど、「月に衝突し、消滅した」と明らかにした。月面着陸前の軌道に移行中、制御不能となったという。
探査機は21日に月の南極付近に着陸予定だった。
```
""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


ユーザー: 下記の文章をJSON形式に変換してください。

```
【速報】ロシア月探査機「ルナ25」が月に衝突 「消滅した」ロスコスモス発表
月に向かっていたロシアの無人探査機「ルナ25」が月に衝突したことが分かった。ロシアの国営宇宙企業ロスコスモスは先ほど、「月に衝突し、消滅した」と明らかにした。月面着陸前の軌道に移行中、制御不能となったという。
探査機は21日に月の南極付近に着陸予定だった。
```
システム: ルナ25の衝突により、月面着陸前の軌道に移行する前に、制御不能となりました。</s>


## Instruction PPO Model

Model: https://huggingface.co/rinna/bilingual-gpt-neox-4b-instruction-ppo

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b-instruction-ppo", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    "rinna/bilingual-gpt-neox-4b-instruction-ppo",
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map="auto",
)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [ ]:
text = """
りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？
""".strip()
inputs = tokenizer(f"ユーザー: {text}\nシステム: 一歩ずつ考えましょう。", return_tensors='pt', add_special_tokens=False)

model.eval()
with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_length=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        top_k=0,
        repetition_penalty=1.1,
        num_beams=1,
        pad_token_id=tokenizer.pad_token_id,
        num_return_sequences=1,
    )

output = tokenizer.decode(output_ids[0], skip_special_tokens=False)
print(output)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ユーザー: りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう?
システム: 一歩ずつ考えましょう。まず、りんごを5つ持っていることから、りんごが5つあることがわかります。次に、りんごを取り除くことで、りんごが5つあることが減るため、りんごが5つあることがなくなります。したがって、答えは0です。</s>


In [ ]:
text = """バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)


ユーザー: バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう?
システム: 答えは、1000-1100=300です。


In [ ]:
text = """
引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)

ユーザー: 引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
システム: def fib(k): return k[:k-1]


In [ ]:
text = """
下記の英語を日本語に翻訳してください。
English: There were 3 apples and 2 oranges. How many fruits were there in total?
""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


ユーザー: 下記の英語を日本語に翻訳してください。
English: There were 3 apples and 2 oranges. How many fruits were there in total?
システム: There were 3 apples and 2 oranges, so there were 3 + 2 = 5 fruits in total.</s>


In [ ]:
text = """
下記の文章を100字以内に要約してください。

```
【速報】ロシア月探査機「ルナ25」が月に衝突 「消滅した」ロスコスモス発表
月に向かっていたロシアの無人探査機「ルナ25」が月に衝突したことが分かった。ロシアの国営宇宙企業ロスコスモスは先ほど、「月に衝突し、消滅した」と明らかにした。月面着陸前の軌道に移行中、制御不能となったという。
探査機は21日に月の南極付近に着陸予定だった。
```
""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


ユーザー: 下記の文章を100字以内に要約してください。

```
【速報】ロシア月探査機「ルナ25」が月に衝突 「消滅した」ロスコスモス発表
月に向かっていたロシアの無人探査機「ルナ25」が月に衝突したことが分かった。ロシアの国営宇宙企業ロスコスモスは先ほど、「月に衝突し、消滅した」と明らかにした。月面着陸前の軌道に移行中、制御不能となったという。
探査機は21日に月の南極付近に着陸予定だった。
```
システム: ロシアが月探査機を月に衝突させたことが、関係者によって明らかになった。</s>


In [ ]:
text = """
下記の文章をJSON形式に変換してください。

```
【速報】ロシア月探査機「ルナ25」が月に衝突 「消滅した」ロスコスモス発表
月に向かっていたロシアの無人探査機「ルナ25」が月に衝突したことが分かった。ロシアの国営宇宙企業ロスコスモスは先ほど、「月に衝突し、消滅した」と明らかにした。月面着陸前の軌道に移行中、制御不能となったという。
探査機は21日に月の南極付近に着陸予定だった。
```
""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


ユーザー: 下記の文章をJSON形式に変換してください。

```
【速報】ロシア月探査機「ルナ25」が月に衝突 「消滅した」ロスコスモス発表
月に向かっていたロシアの無人探査機「ルナ25」が月に衝突したことが分かった。ロシアの国営宇宙企業ロスコスモスは先ほど、「月に衝突し、消滅した」と明らかにした。月面着陸前の軌道に移行中、制御不能となったという。
探査機は21日に月の南極付近に着陸予定だった。
```
システム: [TABLECONTEXT] [TITLE] ルナ25 - 月探査機 [TABLECONTEXT] [TITLE] 月への落下 [TABLECONTEXT] [TITLE] 「月に衝突し、消滅した」 [TABLECONTEXT] [TITLE] 「月に衝突し、消滅した」 [TABLECONTEXT] [TITLE] 「月に衝突し、消滅した」 [TABLECONTEXT] [TITLE] 「月に衝突し、消滅した」 [TABLECONTEXT] [TITLE] 「月に衝突し、消滅した」 [TABLECONTEXT] [TITLE] 「月に衝突し、消滅した」 [TABLECONTEXT] [TITLE] 「月に衝突し、消滅した」 [TABLECONTEXT] [TITLE] 「月に衝突し、消滅した」 [TABLECONTEXT] [TITLE] 「月に衝突し、消滅した」 [TABLECONTEXT] [TITLE] 「月に衝突し、消滅した」 [TABLECONTEXT] [TITLE] 「月に衝突し、消滅した」 [TABLECONTEXT] [TITLE] 「月に衝突し、消滅した」 [TABLECONTEXT] [


In [ ]:
text = """
指示文: あなたは友達ボットです。できるだけユーザーが友情を感じやすいよう接してください。
ユーザー: あー人生の意味って何なんだろうねぇー。
""".strip()

inputs = tokenizer(f"{text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=1.0,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


指示文: あなたは友達ボットです。できるだけユーザーが友情を感じやすいよう接してください。
ユーザー: あー人生の意味って何なんだろうねぇー。
システム: それは、ユーザーが自分自身の人生に対する意味を見いだすことができるかどうかです。</s>
